In [1]:
import numpy as np
import pandas as pd
from sklearn import *
import nltk, datetime
from matplotlib import pylab as plt
import timeit

# Loading all data files
directory = '../data/'
train = pd.read_csv(directory + 'sales_train_v2.csv', parse_dates=['date'], infer_datetime_format=True, dayfirst=True)
test = pd.read_csv(directory + 'test.csv')
submission = pd.read_csv(directory + 'sample_submission.csv')
items = pd.read_csv(directory + 'items.csv')
item_cats = pd.read_csv(directory + 'item_categories.csv')
shops = pd.read_csv(directory + 'shops.csv')

C:\Users\lucas\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\lucas\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
C:\Users\lucas\Anaconda3\lib\site-packages\sklearn\learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20
  DeprecationWarning)


In [2]:
# Removing unnecesary features for this model 
train_clean = train.drop(labels = ['date', 'item_price'], axis = 1)

# Change the item count per day to item count per month by using grouping
train_clean = train_clean.groupby(["item_id","shop_id","date_block_num"]).sum().reset_index()
train_clean = train_clean.rename(index=str, columns = {"item_cnt_day":"item_cnt_month"})

In [3]:
# Data preprocessing cell
num_month = train['date_block_num'].max()
month_list = [i for i in range(num_month + 1)] 

# Shop and item selection
shops = [54] * (num_month + 1) 
items = [22167] * (num_month + 1)

check = train_clean[["shop_id","item_id","date_block_num","item_cnt_month"]]
check = check.loc[check['shop_id'] == 54]
check = check.loc[check['item_id'] == 22167]

months_full = pd.DataFrame({'shop_id': shops, 'item_id': items, 'date_block_num':month_list})

sales_33month = pd.merge(check, months_full, how='right', on=['shop_id','item_id','date_block_num'])
sales_33month = sales_33month.sort_values(by=['date_block_num'])
sales_33month.fillna(0.00, inplace=True)

df = sales_33month[['shop_id','item_id','date_block_num','item_cnt_month']].reset_index()
df = df.drop(labels = ['index'], axis = 1)

x, y = df, df.item_cnt_month

In [4]:
from sklearn.cross_validation import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=42)

x_train = x_train.drop(["item_cnt_month"], axis=1)
x_test = x_test.drop(["item_cnt_month"], axis=1)

In [5]:
x_train

,shop_id,item_id,date_block_num
21,54,22167,21
12,54,22167,12
32,54,22167,32
9,54,22167,9
0,54,22167,0
4,54,22167,4
16,54,22167,16
17,54,22167,17
5,54,22167,5
13,54,22167,13


In [6]:
# Reshape the data between -1 and 1 and to 3D
from sklearn.preprocessing import RobustScaler, MinMaxScaler
scaler = RobustScaler()
x_train_scaled = scaler.fit_transform(x_train)

x_train_reshaped = x_train_scaled.reshape((x_train_scaled.shape[0], 1, x_train_scaled.shape[1]))
y_train_reshaped = y_train.as_matrix().reshape(y_train.shape[0], )

x_test_scaled = scaler.fit_transform(x_test)
x_test_reshaped = x_test_scaled.reshape((x_test_scaled.shape[0], 1, x_test_scaled.shape[1]))

C:\Users\lucas\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys


In [7]:
# Model instantiation

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.core import Dropout, Activation

model = Sequential()

model.add(LSTM(15, input_shape=(1, 3), return_sequences=True, activation='tanh'))
model.add(Dense(1))

model.add(Dropout(0.1))
  
model.add(LSTM(33)) 
model.add(Dropout(0.1))

model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adagrad', metrics=['mean_squared_error', 'accuracy'])

model.summary()

C:\Users\lucas\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 1, 15)             1140      
_________________________________________________________________
dense_1 (Dense)              (None, 1, 1)              16        
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 1)              0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 33)                4620      
_________________________________________________________________
dropout_2 (Dropout)          (None, 33)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 34        
Total params: 5,810
Trainable params: 5,810
Non-trainable params: 0
_________________________________________________________________


In [8]:
start = timeit.default_timer()
history = model.fit(x_train_reshaped, y_train_reshaped,  epochs = 100, batch_size = 33, verbose=1, shuffle = False, validation_split = 0.95)
stop = timeit.default_timer()

# Duration
print (stop - start)

Train on 1 samples, validate on 27 samples
Epoch 1/100
1/1 [==============================] - 1s 1s/step - loss: 0.9993 - mean_squared_error: 0.9993 - acc: 0.0000e+00 - val_loss: 3.6150 - val_mean_squared_error: 3.6150 - val_acc: 0.5926
Epoch 2/100
1/1 [==============================] - 0s 3ms/step - loss: 0.9428 - mean_squared_error: 0.9428 - acc: 0.0000e+00 - val_loss: 3.5752 - val_mean_squared_error: 3.5752 - val_acc: 0.5926
Epoch 3/100
1/1 [==============================] - 0s 3ms/step - loss: 0.9010 - mean_squared_error: 0.9010 - acc: 0.0000e+00 - val_loss: 3.5414 - val_mean_squared_error: 3.5414 - val_acc: 0.5926
Epoch 4/100
1/1 [==============================] - 0s 2ms/step - loss: 0.8593 - mean_squared_error: 0.8593 - acc: 0.0000e+00 - val_loss: 3.5092 - val_mean_squared_error: 3.5092 - val_acc: 0.5926
Epoch 5/100
1/1 [==============================] - 0s 3ms/step - loss: 0.8313 - mean_squared_error: 0.8313 - acc: 0.0000e+00 - val_loss: 3.4794 - val_mean_squared_error: 3.4794 -

Epoch 43/100
1/1 [==============================] - 0s 3ms/step - loss: 0.0412 - mean_squared_error: 0.0412 - acc: 1.0000 - val_loss: 2.8493 - val_mean_squared_error: 2.8493 - val_acc: 0.3704
Epoch 44/100
1/1 [==============================] - 0s 3ms/step - loss: 0.3577 - mean_squared_error: 0.3577 - acc: 0.0000e+00 - val_loss: 2.8428 - val_mean_squared_error: 2.8428 - val_acc: 0.3704
Epoch 45/100
1/1 [==============================] - 0s 2ms/step - loss: 0.0293 - mean_squared_error: 0.0293 - acc: 1.0000 - val_loss: 2.8390 - val_mean_squared_error: 2.8390 - val_acc: 0.3333
Epoch 46/100
1/1 [==============================] - 0s 3ms/step - loss: 0.0212 - mean_squared_error: 0.0212 - acc: 1.0000 - val_loss: 2.8357 - val_mean_squared_error: 2.8357 - val_acc: 0.3333
Epoch 47/100
1/1 [==============================] - 0s 3ms/step - loss: 0.0218 - mean_squared_error: 0.0218 - acc: 1.0000 - val_loss: 2.8325 - val_mean_squared_error: 2.8325 - val_acc: 0.3333
Epoch 48/100
1/1 [==================

Epoch 86/100
1/1 [==============================] - 0s 3ms/step - loss: 0.0016 - mean_squared_error: 0.0016 - acc: 1.0000 - val_loss: 2.7875 - val_mean_squared_error: 2.7875 - val_acc: 0.1481
Epoch 87/100
1/1 [==============================] - 0s 3ms/step - loss: 0.0042 - mean_squared_error: 0.0042 - acc: 1.0000 - val_loss: 2.7874 - val_mean_squared_error: 2.7874 - val_acc: 0.1481
Epoch 88/100
1/1 [==============================] - 0s 3ms/step - loss: 3.1882e-04 - mean_squared_error: 3.1882e-04 - acc: 1.0000 - val_loss: 2.7874 - val_mean_squared_error: 2.7874 - val_acc: 0.1481
Epoch 89/100
1/1 [==============================] - 0s 3ms/step - loss: 0.0126 - mean_squared_error: 0.0126 - acc: 1.0000 - val_loss: 2.7877 - val_mean_squared_error: 2.7877 - val_acc: 0.1481
Epoch 90/100
1/1 [==============================] - 0s 2ms/step - loss: 0.0044 - mean_squared_error: 0.0044 - acc: 1.0000 - val_loss: 2.7875 - val_mean_squared_error: 2.7875 - val_acc: 0.1481
Epoch 91/100
1/1 [==============

In [9]:
start = timeit.default_timer()
y_predicted = model.predict(x_test_reshaped)
stop = timeit.default_timer()

print (stop - start)

0.14003093151561163


In [10]:
from sklearn.metrics import mean_squared_error
from numpy import sqrt
rmse = sqrt(mean_squared_error(y_test, y_predicted))
print('Val RMSE: %.3f' % rmse)

Val RMSE: 1.309
